라이브러리
=================

In [30]:

import tensorflow as tf
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import meshplot as mp
import cv2, os, glob, tqdm
import pandas as pd
import keras.backend as K
import skimage.measure
from stl import mesh

기본 설정
============

In [4]:
main_path= 'C:/Users/fhsdn/Desktop/s/'
target_path = f'{main_path}tooth01.nii'
model_path = f'{main_path}Best1.h5'
out_path = f'{main_path}'

resultOutputPath = f'{out_path}result.stl'



In [6]:
#이미지 설정

IMAGE_HEIGHT = 400
IMAGE_WIDTH = 400
IMG_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)

In [7]:
#함수들

def normalizeImageIntensityRange(img):
    img[img < HOUNSFIELD_MIN] = HOUNSFIELD_MIN
    img[img > HOUNSFIELD_MAX] = HOUNSFIELD_MAX
    return (img - HOUNSFIELD_MIN)/HOUNSFIELD_RANGE

def scaleImg(img, height, width):
    return cv2.resize(img, dsize=(width, height), interpolation=cv2.INTER_LINEAR)

def predictVolume(inImg, toBin=True, supervision=True,get_last = False):
    (xMax, yMax, zMax) = inImg.shape
    
    outImg = np.zeros((xMax, yMax, zMax))
    
    cnt = 0.0
    
    cnt += 1.0
    for i in tqdm.tqdm(range(zMax)):
        img = scaleImg(inImg[:,:,i], IMAGE_HEIGHT, IMAGE_WIDTH)[np.newaxis,:,:,np.newaxis]
        prediction = model.predict(img,verbose=0)
        if supervision:
            num_of_output = len(model.output)
            pred_mask = prediction[num_of_output-1] if get_last else sum(prediction) / num_of_output
        else:
            pred_mask = prediction

        outImg[:,:,i] = scaleImg(pred_mask[0,:,:,0], xMax, yMax)
            
    if(toBin):
        outImg[outImg>=0.5] = 1.0
        outImg[outImg<0.5] = 0.0
    return outImg



def dataToMesh(vert, faces):
    mm = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            mm.vectors[i][j] = vert[f[j],:]
    return mm

사용자 정의 손실함수
=======================

In [8]:
def DiceBCELoss(y_true, y_pred,smooth=1e-6):    
    
    y_true_f= K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)

    BCE =  tf.keras.losses.binary_crossentropy(y_true_f, y_pred_f)
    intersection = K.sum(y_true_f * y_pred_f, axis=-1)
    dice_loss = 1 - (2 * intersection + smooth) / (K.sum(y_true_f,-1) + K.sum(y_pred_f,-1) + smooth)
    Dice_BCE = BCE + dice_loss
    
    return Dice_BCE
    
def IoULoss(y_true, y_pred, smooth=1e-6):

    y_pret_f = K.flatten(y_pred)
    y_true_f = K.flatten(y_true)
    
    intersection = K.sum(y_true_f* y_pret_f)
    total = K.sum(y_true_f) + K.sum(y_pret_f)
    union = total - intersection
    
    IoU = (intersection + smooth) / (union + smooth)
    return 1 - IoU


def FocalLoss(y_true, y_pred, alpha=1.0, gamma=2.0):    
    
    y_pret_f = K.flatten(y_pred)
    y_true_f = K.flatten(y_true)
    
    BCE = K.binary_crossentropy(y_true_f, y_pret_f)
    BCE_EXP = K.exp(-BCE)
    focal_loss = K.mean(alpha * K.pow((1-BCE_EXP), gamma) * BCE)
    
    return focal_loss

  
def Focal_IoU(y_true, y_pred):
  return FocalLoss(y_true,y_pred) + IoULoss(y_true, y_pred)

custom_loss = {'DiceBCELoss': DiceBCELoss,'Focal_IoU':Focal_IoU}

3D영상 만들기 - 모델
=======

In [ ]:
imgTargetNii = nib.load(target_path).get_fdata()
HOUNSFIELD_MIN = np.min(imgTargetNii)
HOUNSFIELD_MAX = np.max(imgTargetNii)
HOUNSFIELD_RANGE = HOUNSFIELD_MAX - HOUNSFIELD_MIN
imgTarget = normalizeImageIntensityRange(imgTargetNii)

model = tf.keras.models.load_model(f'{model_path}',custom_objects=custom_loss)
print('Options')
print(f'밝기 값 범위:{HOUNSFIELD_MIN}~{HOUNSFIELD_MAX}')

#예측
print(f'예측시작')
predImg = predictVolume(imgTarget,True)
print('예측 완료')

if os.path.exists(out_path) == False:
    os.mkdir(out_path)



In [29]:
#Convert binary image to mesh

vertices,faces,_,_ = skimage.measure.marching_cubes(predImg)
#mp.plot(vertices, faces)

#Save mesh to file (.stl)
mm = dataToMesh(vertices, faces)
mm.save(resultOutputPath)

print(f'{resultOutputPath}에 저장되었습니다.')

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(219.0, 23…

C:/Users/fhsdn/Desktop/s/result.stl에 저장되었습니다.
